# **Previsão de Pouso do Primeiro Estágio de Foguetes Falcon 9**


A taxa de sucesso do lançamento pode depender de muitos fatores, como massa da carga útil, tipo de órbita entre outras variáveis. Também pode depender da localização e das proximidades de um local de lançamento, ou seja, a posição inicial das trajetórias do foguete.

Nas seções anteriores, visualizamos o conjunto de dados de lançamento da SpaceX usando `matplotlib` e `seaborn`, além de descobrirmos algumas correlações existentes entre o local de lançamento e as taxas de sucesso. Nesta seção, seram realizadas análises visuais interativas usando a biblioteca `Folium`.

### Objetivos

* Marcar os locais de lançamento em um mapa;
* Marcar os lançamentos bem-sucedidos/falhos para cada local no mapa;
* Calcular as distâncias entre um local de lançamento e suas proximidades.

Depois de concluir os objetivos acima, será possível encontrar padrões geográficos sobre os locais de lançamento.

### Importação das bibliotecas necessárias

In [1]:
#!pip3 install folium
#!pip3 install wget
#!pip install folium==0.12.1 #latest version

In [1]:
import folium
import wget
import pandas as pd

import folium.plugins as plugins
# Importa o plugin folium de conjunto de marcadores de mapa
from folium.plugins import MarkerCluster
# Importa o plugin folium de posição do mouse no mapa
from folium.plugins import MousePosition
# Importa o plugin folium de ícones de mapa
from folium.features import DivIcon

### Marcação dos locais de lançamento no mapa

Primeiro adicionamos a localização de cada local de lançamento em um mapa usando as latitudes e longitudes de cada local de lançamentos.

O conjunto de dados a seguir com o nome `spacex_launch_geo.csv` é um conjunto de dados em que foi acrescentado os dados de latitude e longitude de cada local de lançamento.

In [3]:
# Faz o download e lê o conjunto de dados `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.head(5)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [5]:
# Agrupa os dados da coluna local de lançamento e recorta os dados das colunas `Lat(Latitude)`, `Long(Longitude)` e `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


As coordenadas acima são apenas números simples que não podem fornecer informações intuitivas sobre onde estão esses locais de lançamento. Para facilitar a visualização desses locais vamos fixa-los em um mapa.

Para isso, criamos um objeto 'Mapa' do folium, com uma localização inicial central sendo o Centro Espacial Johnson da NASA em Houston, Texas.

In [6]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Usamos o `folium.Circle` para adicionar uma área de destaque no mapa em formato de círculo com um rótulo de texto em uma coordenada específica

In [7]:
# Cria um círculo azul nas coordenadas do NASA Johnson Space Center com um rótulo pop-up mostrando seu nome
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Crie um círculo azul nas coordenadas do NASA Johnson Space Center com um ícone mostrando seu nome
marker = folium.map.Marker(
    nasa_coordinate,
    # Criar um ícone como um rótulo de texto
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Depois de criar a área circular no mapa referente ao NASA Johnson Space Center, adicionamos também outros circulos referentes aos locais de lançamento (`launch_sites`).


In [8]:
# Apresentação inicial do mapa
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Adição do círculo referente ao NASA Johnson Space Center 
circle = folium.Circle(nasa_coordinate, radius=1000, color='red', 
fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Para cada local de lançamento, adicione um objeto folium.Circle com base em seus valores de coordenadas (Lat, Long). Além disso, adicione o nome do site de lançamento como um rótulo pop-up (folium.Popup)
for idx, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    folium.Circle(coordinate, radius=1000, color='#d35400', 
    fill=True).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)

# Adciona um marcador na forma de rótulo de texto
    folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), 
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'], )).add_to(site_map)
    
site_map


Com o mapa criado torna-se mais fácil e eficiente identificar algumas características como:

*   Os locais de lançamento estão localizados próximo a linha do Equador
*   Os locais de lançamento estão localizados bem próximos a costa

### Marcação dos lançamentos bem-sucedidos/falhos de acordo com cada local de lançamento no mapa

Para aprimorar o mapa adicionamos os resultados de lançamento de cada local com o objetivo de ver quais sítios de lançamento têm maiores taxas de sucesso.
Para isso, usaremos a coluna `class` do dataframe `spacex_df` que indica se o lançamento foi bem sucedido ou não.

In [9]:
# Apresenta os 10 últimos registros do dataframe spacex_df
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Criamos então marcadores para todos os registros de lançamento. Para lançamentos bem sucedidos `(class=1)` usamos um marcador verde e para lançamentos falhos `(class=0)` usamos um marcador vermelho. 



Observação: Um lançamento acontece apenas em um dos quatro locais de lançamento, o que significa que muitos registros de lançamento terão exatamente a mesma coordenada. Agrupamentos de marcadores são uma boa maneira de simplificar um mapa que contém muitos marcadores com a mesma coordenada.

In [10]:
# Cria um objeto `MarkerCluster` para agrupar os marcadores
marker_cluster = MarkerCluster()

Posteriormente, criamos uma nova coluna no dataframe `launch_sites` chamada `marker_color` para armazenar as cores do marcador com base no valor `class`

In [11]:
# Aplique uma função para verificar o valor da coluna `class`
# Para class=1, o valor marker_color será verde
# Para class=0, o valor marker_color será vermelho

spacex_df['marker_color'] = pd.DataFrame(['green' if cls == 1 else 'red' for cls in spacex_df['class']])

spacex_df.tail(10)

# Para ver o dataframe com a nova coluna marker_color
#spacex_df['marker_color']

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [12]:
# Função que atribui cor ao resultado do lançamento
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


Para cada resultado de lançamento no dataframe `spacex_df` adicionamos um marcador `folium.Marker` ao agrupamento de marcadores `marker_cluster`

In [13]:
# Adiciona o agrupamento de marcadores marker_cluster ao mapa site_map
site_map.add_child(marker_cluster)

# Para cada linha no dataframe spacex_df cria um objeto Marker com sua coordenada, personalizndo a 
# propriedade do ícone do Marcador para indicar se o lançamento foi bem sucedida ou não, 
# por exemplo, icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # Create and add a Marker cluster to the site map
    # Cria e adiciona um cluster de marcador ao mapa dos locais de lançamento
    marker = folium.Marker(location = [record['Lat'], record['Long']],
                            icon = folium.Icon(color = 'white', icon_color = record['marker_color']),
                            popup = 'Successed' if record['class'] == 1 else 'Failed'
                            )
    marker_cluster.add_child(marker)

site_map


Os circulos amarelos e verde representam o agrupamento de marcadores e ao clicar nestes é possível ver todos os marcadores que foram agrupados, sendo estes referentes aos sucesso ou falha dos lançamentos

### Calculo das distâncias entre o local de lançamento e suas proximidades

Primeiro adicionamos um `MousePosition` no mapa para obter as coordenadas do cursor do mouse sobre um ponto no mapa. Com isso, é possível pode encontrar facilmente as coordenadas de quaisquer pontos de interesse no mapa, como ferrovias, rodovias, etc

In [20]:
# Adiciona o MousePosition para obter as coordenadas (Lat, Long) ao passar o mouse no mapa
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

# As coordenadas aparecem no canto superior direito do mapa!!

Calcula-se então a distância entre dois pontos no mapa com base em seus valores `Lat` e `Long` usando o seguinte método:

In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # Raio aproximado da terra em km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

### Calculo das distâncias entre um ponto no litoral e o local de lançamento mais próximo

A seguir tem-se um exemplo de cálculo da distância distância entre um ponto no litoral (por MousePosition) e um local de lançamento.

In [16]:
# Exemplo de ponto no litoral: Lat: 28.56367 Long: -80.57163
# Exemplo de local de lançamento: Lat: 34.63293 Long: -120.611
coastline_lat = 34.63667
coastline_lon = -120.62585
launch_site_lat = 34.63293
launch_site_lon = -120.611
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

1.4212985693626217

Após obter a distância, criamos um marcador `folium.Marker` para apresentar a distância entre os pontos anteriormente calculados

In [21]:
# Cria e adiciona um marcador no ponto de litoral mais próximo selecionado no mapa
# Exibe a distância entre o ponto do litoral e o local de lançamento usando as propriedades de ícone
distance_marker = folium.Marker(
   [coastline_lat, coastline_lon],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )
site_map.add_child(distance_marker)

Vamos desenhar também uma `PolyLine` entre o local de lançamento até o ponto do litoral para melhorar a visualização no mapa

In [23]:
# Cria um objeto `folium.PolyLine` usando as coordenadas do litoral e a coordenada do local de lançamento
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines=folium.PolyLine(locations=coordinates, weight=3)
site_map.add_child(lines)
site_map

### Calculo das distâncias entre a ferrovia mais próxima e o local de lançamento

O símbolo de uma ferrovia no mapa é apresentado da seguinte forma:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


In [25]:
# Cria um marcador com o valor da distância entre o local de lançamento e a ferrovia mais próxima
# Desenha uma linha entre o marcador e o local de lançamento
railway_lat = 34.63522
railway_lon = -120.62413
launch_site_lat = 34.63293
launch_site_lon = -120.611
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
distance_railway

distance_marker_railway = folium.Marker(
   [railway_lat, railway_lon],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
       )
   )
site_map.add_child(distance_marker_railway)

coordinates1 = [[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]]
lines1=folium.PolyLine(locations=coordinates1, weight=3)
site_map.add_child(lines1)
site_map

### Calculo das distâncias entre a rodovia mais próxima e o local de lançamento

O símbolo de uma rodovia no mapa é apresentado da seguinte forma:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


In [26]:
highway_lat = 34.63303
highway_lon = -120.62366
launch_site_lat = 34.63293
launch_site_lon = -120.611
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
distance_highway

distance_marker_highway = folium.Marker(
   [highway_lat, highway_lon],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
       )
   )
site_map.add_child(distance_marker_highway)

coordinates2 = [[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]]
lines2=folium.PolyLine(locations=coordinates2, weight=3)
site_map.add_child(lines2)
site_map

### Calculo das distâncias entre a cidade mais próxima e o local de lançamento

A cidade mais próxima representada pelo seu nome conforme a seguir:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


In [27]:
city_lat = 34.63886
city_lon = -120.45805
launch_site_lat = 34.63293
launch_site_lon = -120.611
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)
distance_city

distance_marker_city = folium.Marker(
   [city_lat, city_lon],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
       )
   )
site_map.add_child(distance_marker_city)

coordinates3 = [[launch_site_lat, launch_site_lon], [city_lat, city_lon]]
lines3=folium.PolyLine(locations=coordinates3, weight=3)
site_map.add_child(lines3)
site_map

Depois de calcular as distâncias e traçar as linhas representativas das distâncias, pode-se obter as seguintes informações:

* As ferrovias mais próximas aos locais de lançamento;
* As costas mais próxima aos locais de lançamento;
* As rodovias mais próximas aos locais de lançamento;
* As cidades mais próximas aos locais de lançamento.